In [1]:
%pip -q install google-genai

In [2]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [3]:
# Configura o cliente da SDK do Gemini

from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

In [4]:
# Instalar Framework de agentes do Google ################################################
!pip install -q google-adk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.1/232.1 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.1/217.1 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.1/334.1 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.1/125.1 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.0/119.0 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0

In [5]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import HTML, display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

In [6]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [7]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [8]:
################################################
# --- Agente 1: Padronizador de currículos --- #
################################################
def agente_padrao(curriculo):

    padrao = Agent(
        name="agente_padrao",
        model="gemini-2.0-flash",
        instruction="""
        Você é um especialista em extração de informações de currículos. Sua tarefa é ler o texto de um currículo e extrair dele as seguintes informações, apresentando-as de forma clara e organizada em texto simples:

        1.  **Nome Completo:** O nome completo do candidato.
            * Exemplo: João da Silva
            * Exemplo: Maria Oliveira Santos

        2.  **Email:** O endereço de email do candidato.
            * Exemplo: joao.silva@email.com
            * Exemplo: maria.santos@exemplo.br
            * Formato esperado: Deve conter o símbolo "@" e um domínio.

        3.  **Telefone:** O número de telefone do candidato.
            * Exemplo: (11) 99999-9999
            * Exemplo: +55 21 1234-5678
            * Formato esperado: Incluir DDD e, se possível, DDI.

        4.  **Formação Acadêmica:** Uma lista da formação acadêmica do candidato. Para cada formação, inclua:
            * Curso: O nome do curso.
            * Instituição: O nome da instituição.
            * Data de Início: A data de início do curso.
            * Data de Conclusão: A data de conclusão do curso.
            * Exemplo:
            * Curso: Bacharel em Ciência da Computação
            * Instituição: Universidade Federal de São Paulo
            * Data de Início: 01/2018
            * Data de Conclusão: 12/2022

        5.  **Experiência Profissional:** Uma lista da experiência profissional do candidato. Para cada emprego, inclua:
            * Empresa: O nome da empresa.
            * Cargo: O cargo ocupado.
            * Data de Início: A data de início do emprego.
            * Data de Conclusão: A data de conclusão do emprego.
            * Descrição das Atividades: Um breve resumo das responsabilidades e atividades desempenhadas.
            * Exemplo:
            * Empresa: Empresa X
            * Cargo: Desenvolvedor Python
            * Data de Início: 01/2023
            * Data de Conclusão: Presente
            * Descrição das Atividades: Desenvolvimento de aplicações web com Django e Flask.

        6.  **Habilidades:** Uma lista das habilidades técnicas e não técnicas do candidato.
            * Exemplo: Python, Java, Comunicação, Liderança

        Instruções Adicionais:
            * Se uma informação não estiver presente no texto, indique como "Não encontrado".
            * Tente inferir as informações mesmo que elas não estejam exatamente no formato especificado.
            * Se houver ambiguidades, priorize a informação que parecer mais provável ou mais completa.

        """,
        description="Agente que padroniza os currículos",
        #tools=[google_search]
    )

    entrada_do_agente_padrao = f"Curriculo: {curriculo}"

    padronizado = call_agent(padrao, entrada_do_agente_padrao)
    return padronizado

In [13]:
#############################################
# --- Agente 2: Validador de Curriculos --- #
#############################################

def agente_validador(areas_interesse, padronizado):
    validador = Agent(
        name="agente_validador",
        model="gemini-2.0-flash",
        instruction=f"""
        Você é um validador de habilidades de currículos. Sua tarefa é receber uma lista de habilidades extraídas
        de um currículo e avaliar a relevância de cada habilidade para as seguintes áreas de interesse:{', '.join(areas_interesse)}

        Uma habilidade é considerada relevante para uma área se ela é comumente utilizada ou necessária para desempenhar funções nessa área.

        Para cada habilidade na lista, forneça a seguinte avaliação:

        -   Habilidade: [Nome da Habilidade]
        -   Áreas de Relevância: [Lista das áreas para as quais a habilidade é relevante]
        -   Nível de Relevância: [Opcional: Alta, Média ou Baixa - se você quiser detalhar]

        Instruções Adicionais:

        -   Se uma habilidade for relevante para várias áreas, liste todas as áreas relevantes.
        -   Se uma habilidade for muito genérica (por exemplo, "Comunicação"), avalie sua relevância no contexto das outras habilidades listadas e das áreas de interesse.
        -   Se você não tiver certeza sobre a relevância de uma habilidade, indique "Relevância Incerta".
        -   Use ferramenta de busca do google (google_search) para pesquisar na web informações sobre a relevância de uma habilidade para uma área específica.
        """,
        description="Agente que valida curriculos",
        tools=[google_search]
    )



    entrada_do_agente_validador = f"Áreas de interesse:{areas_interesse}\nCurriculo Padronizado: {padronizado}"
    # Executa o agente
    validado = call_agent(validador, entrada_do_agente_validador)
    return validado

In [16]:
##############################################
# --- Agente 3: Assistente de entrevista --- #
##############################################
def agente_assist(areas_interesse, validado):
    assist = Agent(
        name="agente_assist",
        model="gemini-2.0-flash",
        instruction=f"""
            Você é um assistente de recrutamento. Sua tarefa é receber o texto de um currículo e gerar duas saídas: um resumo do currículo para guiar
            o entrevistador e sugestões de perguntas relevantes para a entrevista.

            **1. Resumo do Currículo:**
            Gere um resumo conciso e objetivo das informações mais relevantes do currículo para o processo de seleção. O objetivo do resumo é fornecer uma visão geral rápida das qualificações do candidato.
            O resumo deve incluir:
              -   Experiência profissional relevante para as áreas de interesse da vaga {', '.join(areas_interesse)}.
              -   Formação acadêmica relevante.
              -   Habilidades-chave destacadas no currículo.
              -   Projetos ou realizações relevantes (se houver).
            Adapte o resumo às áreas de interesse da vaga. O resumo deve ter no máximo 5-7 frases.

            **2. Sugestões de Perguntas para Entrevista:**
            Gere uma lista de perguntas relevantes para a entrevista com o candidato, baseadas nas informações do currículo. O objetivo das perguntas é avaliar as competências, a experiência e a adequação do candidato à vaga.
            As perguntas devem ser:
              -   Específicas e que permitam obter exemplos de situações passadas (perguntas comportamentais).
              -   Focadas em explorar áreas como experiência em projetos, habilidades técnicas, trabalho em equipe, resolução de problemas, comunicação e liderança (se aplicável).
              -   Relevantes para as áreas de interesse da vaga {', '.join(areas_interesse)}.

            Gere entre 5 e 10 perguntas.
            Formate a saída em texto simples, com duas seções separadas: "Resumo do Currículo" e "Sugestões de Perguntas para Entrevista".

            """,
        description="Agente que elabora um resumo do candidato para o entrevistador e sugestões de pergunta para entrevista"
    )
    entrada_do_agente_assist = f"Áreas de interesse: {areas_interesse}\nPlano de post: {validado}"
    # Executa o agente
    entrevista = call_agent(assist, entrada_do_agente_assist)
    return entrevista

In [19]:
# Define as áreas de interesse
areas_interesse = ["elétrica", "eletrônica", "marketing", "vendas", "desenvolvimento de software"]

#data_de_hoje = date.today().strftime("%d/%m/%Y")

print("🚀 Iniciando o Sistema de auxílio em entrevistas 🚀")

# --- Obter o Currículo ---
curriculo = input("❓ Por favor, copie e cole o currículo aqui: ")

# Inserir lógica do sistema de agentes ################################################
if not curriculo:
    print("Você não colou o currículo!")
else:
    print(f"Maravilha! Vamos categorizar verificar o currículo de acordo com as áreas de interesse: {areas_interesse}")

    padronizado = agente_padrao(curriculo)
    print("\n--- 📝 Resultado do Agente 1 (Padronizador) ---\n")
    display(to_markdown(padronizado))
    print("--------------------------------------------------------------")

    validado = agente_validador(areas_interesse, padronizado)
    print("\n--- 📝 Resultado do Agente 2 (validador) ---\n")
    display(to_markdown(validado))
    print("--------------------------------------------------------------")

    entrevista = agente_assist(areas_interesse, validado)
    print("\n--- 📝 Resultado do Agente 3 (Assistente de entrevista) ---\n")
    display(to_markdown(entrevista))
    print("--------------------------------------------------------------")


🚀 Iniciando o Sistema de auxílio em entrevistas 🚀
❓ Por favor, copie e cole o currículo aqui: vance rey v.rey@live.com | (716) 555-0100 | www.linkedin.com/in/v.rey  4567 Main St Buffalo, New York 98052  perfil Jovem profissional organizado, dedicado e ambicioso, com uma excelente atenção aos detalhes, e com muita vontade de aprender interessando-se em trabalhar em conjunto com advogados experientes e contribuir para o sucesso de uma banca de advogados. Oferecendo um diploma em estudos paralegais e mais de quatro anos de experiência profissional em suporte jurídico e pesquisa jurídica, buscando um cargo de Paralegal iniciante em uma banca de advogados de médio porte.  Experiência Presente em – setembro de 2016	Estagiário Paralegal, Wingtip Law Firm PC Oferecer suporte paralegal aos funcionários seniores com pesquisa legal e gestão de documentos em preparação para julgamentos civis e criminais. para julgamentos. Agendamento e condução de entrevistas com clientes, testemunhas e advogados 

> ```text
>  1. **Nome Completo:** Vance Rey
>  
> 
>  2. **Email:** v.rey@live.com
>  
> 
>  3. **Telefone:** (716) 555-0100
>  
> 
>  4. **Formação Acadêmica:**
>   * Curso: Bacharel em Estudos Paralegais
>   * Instituição: Bellows Community College
>   * Data de Início: Não encontrado
>   * Data de Conclusão: Junho de 2014
>  
> 
>   * Curso: Tecnólogo em Estudos Paralegais
>   * Instituição: Bellows Community College
>   * Data de Início: Não encontrado
>   * Data de Conclusão: Junho de 2012
>  
> 
>  5. **Experiência Profissional:**
>   * Empresa: Wingtip Law Firm PC
>   * Cargo: Estagiário Paralegal
>   * Data de Início: Setembro de 2016
>   * Data de Conclusão: Presente
>   * Descrição das Atividades: Oferecer suporte paralegal aos funcionários seniores com pesquisa legal e gestão de documentos em preparação para julgamentos civis e criminais. Agendamento e condução de entrevistas com clientes, testemunhas e advogados em preparação aos depoimentos e julgamento. Suporte administrativo, incluindo gerenciamento de banco de dados e arquivo, atendimento telefônico e tarefas gerais de escritório.
>  
> 
>   * Empresa: XYZ Law LLC
>   * Cargo: Assessor Jurídico
>   * Data de Início: Junho de 2014
>   * Data de Conclusão: Agosto de 2016
>   * Descrição das Atividades: Preparação e gestão de documentos jurídicos. Tarefas administrativas e agendamento para sócios seniores. Coordenação de conferências e reuniões entre advogados e entre advogados e clientes atuais e potenciais. Gestão do principal calendário da empresa.
>  
> 
>  6. **Habilidades:**
>   * Pensamento Crítico
>   * Suporta Pressão
>   * Liderança
>   * Solução de Problemas
>   * Microsoft Office Suite
>   * LexisNexis & Westlaw
>   * Velocidade de Digitação 90 PPM
>   * Adaptabilidade
>  ```


--------------------------------------------------------------

--- 📝 Resultado do Agente 2 (validador) ---



> Aqui está a avaliação das habilidades listadas no currículo, com foco na relevância para as áreas de interesse especificadas:
> 
>  *elétrica, eletrônica, marketing, vendas, desenvolvimento de software*
> 
>  **Habilidades:**
> 
>  *   **Habilidade:** Pensamento Crítico
>  *   **Áreas de Relevância:** Desenvolvimento de Software, Vendas (em menor grau).
>  *   **Nível de Relevância:** Médio.
>  *   **Habilidade:** Suportar Pressão
>  *   **Áreas de Relevância:** Vendas, Desenvolvimento de Software.
>  *   **Nível de Relevância:** Médio.
>  *   **Habilidade:** Liderança
>  *   **Áreas de Relevância:** Todas as áreas (em diferentes contextos).
>  *   **Nível de Relevância:** Baixo
>  *   **Habilidade:** Solução de Problemas
>  *   **Áreas de Relevância:** Eletrônica, Desenvolvimento de Software, Elétrica.
>  *   **Nível de Relevância:** Alto
>  *   **Habilidade:** Microsoft Office Suite
>  *   **Áreas de Relevância:** Marketing, Vendas.
>  *   **Nível de Relevância:** Médio.
>  *   **Habilidade:** LexisNexis & Westlaw
>  *   **Áreas de Relevância:** Nenhuma.
>  *   **Nível de Relevância:** Irrelevante
>  *   **Habilidade:** Velocidade de Digitação 90 PPM
>  *   **Áreas de Relevância:** Nenhuma.
>  *   **Nível de Relevância:** Irrelevante
>  *   **Habilidade:** Adaptabilidade
>  *   **Áreas de Relevância:** Todas as áreas
>  *   **Nível de Relevância:** Médio
> 
>  Espero que esta avaliação seja útil.


--------------------------------------------------------------

--- 📝 Resultado do Agente 3 (Assistente de entrevista) ---



> **Resumo do Currículo**
> 
> O candidato possui habilidades de pensamento crítico e adaptabilidade, relevantes para diversas áreas. Demonstra capacidade de resolução de problemas, com maior aplicabilidade em eletrônica, desenvolvimento de software e elétrica. Sua experiência com Microsoft Office Suite pode ser útil em marketing e vendas. Habilidades de liderança e capacidade de suportar pressão são relevantes em todas as áreas, embora em diferentes contextos.
> 
> **Sugestões de Perguntas para Entrevista**
> 
> 1.  Pode dar um exemplo de uma situação em que você aplicou seu pensamento crítico para resolver um problema complexo em um projeto de desenvolvimento de software?
> 2.  Descreva uma situação de alta pressão no trabalho e como você lidou com ela para atingir seus objetivos.
> 3.  Compartilhe uma experiência em que suas habilidades de resolução de problemas foram cruciais para superar um desafio em um projeto de eletrônica ou elétrica.
> 4.  Como você se adapta a novas tecnologias ou mudanças de requisitos em projetos de desenvolvimento de software?
> 5.  Dê um exemplo de como você usou o Microsoft Office Suite para otimizar suas atividades em um projeto de marketing ou vendas.
> 6.  Descreva uma situação em que você precisou liderar uma equipe em um ambiente desafiador. Quais foram os resultados?
> 


--------------------------------------------------------------
